In [1]:
import os

# Set the path to your JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\User\\Downloads\\planar-night-313706-10bb94729179.json"

In [2]:
from google.cloud import vision

try:
    client = vision.ImageAnnotatorClient()
    print("Credentials are properly set!")
except Exception as e:
    print(f"Error: {e}")


Credentials are properly set!


In [3]:
from enum import Enum
from google.cloud import vision
from PIL import Image, ImageDraw

# Define feature types for bounding box extraction
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

def draw_boxes(image, bounds, color):
    """Draws a border around the image using the hints in the vector list.

    Args:
        image: the input image object.
        bounds: list of coordinates for the boxes.
        color: the color of the box.

    Returns:
        An image with colored bounds added.
    """
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        draw.polygon(
            [
                bound.vertices[0].x, bound.vertices[0].y,
                bound.vertices[1].x, bound.vertices[1].y,
                bound.vertices[2].x, bound.vertices[2].y,
                bound.vertices[3].x, bound.vertices[3].y,
            ],
            outline=color
        )
    return image

def get_document_bounds(image_file, feature):
    """Finds the document bounds given an image and feature type.

    Args:
        image_file: path to the image file.
        feature: feature type to detect.

    Returns:
        List of coordinates for the corresponding feature type.
    """
    client = vision.ImageAnnotatorClient()

    bounds = []

    with open(image_file, "rb") as img_file:
        content = img_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    # Collect specified feature bounds
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if feature == FeatureType.SYMBOL:
                            bounds.append(symbol.bounding_box)
                    if feature == FeatureType.WORD:
                        bounds.append(word.bounding_box)
                if feature == FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)
            if feature == FeatureType.BLOCK:
                bounds.append(block.bounding_box)

    return bounds

def render_doc_text(filein, fileout=None):
    """Outlines document features (blocks, paragraphs, and words) on the image.

    Args:
        filein: path to the input image.
        fileout: path to the output image. If None, displays the image.
    """
    image = Image.open(filein)

    # Draw bounding boxes for each feature type
    bounds = get_document_bounds(filein, FeatureType.BLOCK)
    draw_boxes(image, bounds, "blue")
    bounds = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, "red")
    bounds = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, "yellow")

    if fileout:
        image.save(fileout)
        print(f"Processed image saved to {fileout}")
    else:
        image.show()

# Example Usage in Jupyter Notebook
# Define the input and output file paths
input_file = "C:/Users/User/Downloads/page_1_thumb_large.jpg"
output_file = "C:/Users/User/Downloads/output_image.jpg"

# Call the function
render_doc_text(input_file, output_file)


PermissionDenied: 403 This API method requires billing to be enabled. Please enable billing on project #949533424066 by visiting https://console.developers.google.com/billing/enable?project=949533424066 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry. [links {
  description: "Google developers console billing"
  url: "https://console.developers.google.com/billing/enable?project=949533424066"
}
, reason: "BILLING_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "vision.googleapis.com"
}
metadata {
  key: "consumer"
  value: "projects/949533424066"
}
]